# Year 1 Semester 2 Research Diary

------------------------------------------------------

## Until 02/08/2017

The produced code is able to create white noise and convolute it with CMB data. The resulting map is close to what is observed. Efforts to recreate a scan strategy and dissect the map into bolometer signals have been made. However when recompressed into a map there is a substantial difference to the original map, indicating an error somewhere during the dissection and reconstrusction.

The code to create the CMB map works in the following way:

1. Define map parameters. We choose a square map of 2048" sidelength.
1. Read in CMB data. Specifically target $l$ and $D_l$ values, which are the first and second column of the provided in the file *for_lennart/plik_plus_r0p01_highell_lensedtotCls.dat* respectively. Then use $C_l = \frac{2 \pi D_l}{l(l+1)}$ to get $C_l$ values for further analysis.
1. Turn obtained $C_l$ values into a 2D array.
    1. Create wavevectors along each dimension for provided map parameters.
    1. Tile wavevectors to get 2D versions.
    1. Use $C_l = \sqrt{k_x^2+k_y^2}$ to populate two dimensional $C_l$ array.
1. Create noise in Fourier space.
1. Add signal via $\mathrm{CMB} = \sqrt{C_l \frac{df}{2}} \times (\mathrm{realnoise} + i \times \mathrm{imaginarynoise})$
1. Inverse FFT to get the real map, but only keep a quarter

The code produces CMB maps that look like the following. Note that creating the 2D version of $C_l$ is computationally expensive and hence the array was only computed once, then saved and is now loaded. Since this is purely the CMB signal and does not contain noise yet this is adequate.

<img src = "../cosmologydigitisation/Plots/21082017/cmbnoisefourier.png">

## 03/08/2017

Meeting with Christian to discuss progress. Guidance on when to implement noise provided: either at the bolometer stage or when creating the map from its Fourier Transform, but not both.

For the next meeting aim to:
* Be able to recreate maps accurately, after decomposing them into bolometer signals and recombining them.
* Check Parseval's Theorem.

Agreed to schedule meetings more regularly and set clearer expectations.

## 08/08/2017 - 09/08/2017

Data intensive astronomy workshop ADACS.

## 10/08/2017

Completed Version Control setup on personal laptop and university computer.

# 12/08/2017

## Parseval's theorem

We would like to confirm Parseval's theorem, which states that power is conserved when moving between real and frequency space. Expressed mathematically:

$$\sum_n |x(n)|^2 = \frac{1}{N} \sum_n |X(n)|^2,$$

where $X(n)$ is the discrete Fourier transform of $x(n)$, both of length $N$. In our case we must carry out the summation over the entire image, i.e. two dimensions.

Straightforward implementation of this confirms this for the CMB map. We immediately check this after applying the inverse Fourier transform. Confirmation of the above is less trivial for the finished map, as we cut the inverse Fourier transform output into a quarter. To check that Parseval's theorem still holds we draw many noise realisations, apply the inverse Fourier transform and compare the power to their corresponding maps in frequency space.



```
psvindex = range(100)
powdiff = []

for i in psvindex:
    realpart = factor * normal(size=(pixelnumber, pixelnumber))
    imagpart = factor * normal(size=(pixelnumber, pixelnumber))
    cmbnoisefreqspace = (realpart + 1j * imagpart)
    cmbnoisemap = fft.ifft2(cmbnoisefreqspace)[0:int(pixelnumber/2), 0:int(pixelnumber/2)]
    psvfreq = real(sum(cmbnoisefreqspace * conjugate(cmbnoisefreqspace)))
    psvreal = real(sum(cmbnoisemap * conjugate(cmbnoisemap))) * 4 * pixelnumber**2
    powdiff.append((psvfreq-psvreal)/psvfreq)
```

We define $\Delta P = (P_f-P_r)/P_f $ where $P$ indicates power and $f$ and $r$ Fourier and real space respectively. We plot $\Delta P$ and show mean and standard deviation thereof below.

<img src = "../cosmologydigitisation/Plots/12082017/relativepowerdifference.png">

From the plot we see that the powers are consistent over many iterations, albeit the crude cutting of the real space map. We note that we must perform this check before throwing away the imaginary component of the real space map.

We now check if power is conserved in Srini's powerspectrum code. This is not the case. We spot that the Fourier transform of the image is multiplied by the radial pixel resolution, which changes the power. We remove this extra factor and observe that Parseval's theorem is restored.

To check whether power is conserved during the radial averaging process we must retain information about the number of pixels in each radial bin. We then check Parseval's theorem by multiplying each radial power by its bincount and then summing the results. We achieve a result that is appreciably close to the original power, but nevertheless different. Redefining $\Delta P = (P_0-P_r)/P_0$ and drawing 100 realisations we observe the behaviour displayed below.

<img src = "../cosmologydigitisation/Plots/12082017/powerdiffradial.png">

The only explanation we can think of is that this is an effect due to lying circular bins over a rectangular image.

## Recreating Scan Strategies appropriately

We have previously found inconsistencies between a map and a copy of it, which was broken down into bolometer signals and then recompressed. We investigate this difference now.

Recreation and Compression code runs extremely slow on laptop. Unsure how to improve this. We need to iterate over every pixel in the map to break it up into bolometer signals. We need to iterate over all bolometer signals to compress them correctly into the map again. We are unsure if taking the resolution down is appropriate, since it may not capture all CMB features appropriately. To check if the code is correct we take the number of bolometer signals that correspond to a map pixel down from 4000 to 120. This is done by changing the readout frequency from $500\mathrm{Hz}$ to $6\mathrm{Hz}$. We then formulate $\Delta M = (M_0-M_r)/M_0$ where $M_0$ are elements of the original map and $M_r$ are elements of the map that has been expanded and compressed. The result is distinctly non-zero. However, $\Delta M$ is very small and it appears to look like white noise. We check this by calculating its powerspectrum, which confirms this intuition. The relevant plots are shown below. Furthermore we confirm that the average of $\Delta M$ is consistent with zero within one standard deviation.

We therefore have confidence that the code written is correct. Perhaps the error experienced was due to including additional noise when recreating the scan strategy. Without doing this we are however unsure how a faithful digitisation would work that operates at timestream level.

<img src = "../cosmologydigitisation/Plots/12082017/mapdiff.png">

<img src = "../cosmologydigitisation/Plots/12082017/mapdiffpower.png">

# 15/08/2017

Meeting with Christian. Discussed progress on Parseval's theorem and recreating the scan strategy. On Scan strategy everything's good, we're limited by `float64` precision. On Parseval's theorem a little puzzled by the difference induced when doing the radial averaging. Some issues are:

* Power leaking into higher modes
* Not all data points are hit by circular apertures

For the next meeting adjust:

* Mean of the Map = 0 $\checkmark$
* Check if apertures cover all of the map
$\checkmark$ <font color = "Grey"> They did not. This was due to the maximum bin not being set correctly. It used the maximum of the 1D wavevector, rather than the maximum of $\sqrt{l_x^2+l_y^2}$. Adjusted this in a hacky way in Srini's code. After adjusting this Parseval's theorem is restored in the radial averaging.</font>
* Apply a mask to the fft map and zero pad, account for this in PS by multiplying it by $1/\mathrm{avg}(\mathrm{mask}^2)$
$\checkmark$ <font color = "Grey"> Wrote a zeropad and a cosine mask function in python. cos mask is written as array multiplication to save time.</font>
* Change noise to be induced at the time stream level, not in the first fourier transform instance (Noise is added by addition)
    * Began work on this, creating the noise and adding it is trivial, but the raw cmb map does not look right. Powerspectrum doesnt look like it should either. But this is not fault of the powerspectrum code. The PS for the raw CMB image and one recreated from the tod without adding noise have the same PS. Therefore we must look at the creation of the raw cmb image.
    * whats going on with the top left corner in the pure cmb map? Should this be the centre, but fftshift? on the $k_{x, y}$ values when creating CL2D maybe?
    * Does not seem mathematically trivial that the two stages of noise addition should be equivalent
* Increase map size to 20x20 degrees $\checkmark$ <font color = "Grey">Actually have a 34x34 degree field at first, with 2 arcmin resolution.</font>
* Compare powerspectrum in and powerspectrum out. Recreation: $\checkmark$ <font color = "Grey">Leave rest of analysis until noise is sorted out.</font>

$\rightarrow$ Implementing the noise is challenging. While it seems conceptually logical that it should not matter whether we add noise in frequency or real space, they are not mathematically interchangable. The matching operation in real space is convolution. Perhaps for white noise however the convolution and addition are equivalent? This does not seem right however. This should be discussed during the next meeting. Until then, the relevant plots are below.

IFFT of the pure CMB data, i.e. without noise.
<img src = "../cosmologydigitisation/Plots/21082017/cmbrealspace.png">
Noise via addition.
<img src = "../cosmologydigitisation/Plots/21082017/cmbnoiseaddedinreal.png">
Noise via concolution.
<img src = "../cosmologydigitisation/Plots/21082017/cmbnoisebyconvolution.png">

We notice that there are some artifacts to the convolution (stringiness), but it matches the desired features for a mock observation.

Attempted at recreating the input Powerspectrum. Unsure whether to use $P_k$ or $P_k k(k+1)$. Neither shapes look correct.

<img src = "../cosmologydigitisation/Plots/21082017/pspk.png">
<img src = "../cosmologydigitisation/Plots/21082017/pscl.png">

## 25/08/2017

Meeting with Christian. The mock observations is the addition of two gaussian random fields. The CMB and a Noise field. So the CMBNoise field must be added to another random field. This addition can either take place in real space or Fourier space. For good comparison we want to add 1 observed CMB map to N Noise maps to produce N observations. Then coadd (average, dont worry about weighting for now) the N produced maps and calculate the powerspectrum. When adding the noise we can potentially digitise.

* How to set average & standard deviation of added noise? For now set average to 0 and standard deviation to the standard deviation of the CMB map.
* Wrote code to create a CMB map, break it up into tod, create N mockobservations with added noise (can digitise here), then recompress into map, compare powerspectrum
* Noticed white noise is not being displayed appropriately by powerspectrum code, this is due to applying the cosine mask

The CMB maps PS is compared to the average of 10 maps with added noise below.

<img src = "../cosmologydigitisation/Plots/31082017/pscompare.png">


## 31/08/2017

Meeting with Christian. Needed to Apply cosine mask and zeropadding in real space, not Fourier space. This fixed the strange appearance of the white noise. For the next meeting aim to:

* Fix scale of noise. Want $6\mu \mathrm{K}$ per pixel. $\checkmark$ Implemented this such that this corresponds to the variance of the noise added to the cmb at timestream level. Does this change if we move to N obervations?
* Substract CMB map and the averaged noise map to see the difference in real space $\checkmark$ This is done for 50 obervations and the result shown below

<img src = "../cosmologydigitisation/Plots/05092017/cmbnoisecomparemap.png">

* Plot Powerspectra $\checkmark$ this is again done for 50 noise maps

<img src = "../cosmologydigitisation/Plots/05092017/cmbnoisecompareps.png">

The difference has a mean of 0 within one $\sigma$.

* Digitise and compare powerspectra $\checkmark$ Digitised 1Bit, 10 observations, comparison is below

<img src = "../cosmologydigitisation/Plots/05092017/ps1bit10obs.png">

Difference may not be identical with 0. Same calculation but now plotting $p_k \times k (k+1)$ reveals a stronger discrepancy at intermediate modes.

<img src = "../cosmologydigitisation/Plots/05092017/pscl1bit10obs.png">

Repeated Digitisation for 5 maps and half maximum 2 bit digitisation. Result is below. Approx 3m runtime. Scales roughly with calls to digitisation.

<img src = "../cosmologydigitisation/Plots/05092017/ps2bithm5obs.png">

The difference has a mean of 0 within one $\sigma$.

* Use profiler to analyse where the code needs to be improved (see cprofiler, pstats for this) $\checkmark$ Used this to reduce number of calls to normal(), decreased cumulative time for one noise map without digitisation from $2.875\to 1.524$ Also removed use of deepcopy() Can look into a better approach for normalising the noise, but that has very little effect to other calculations
* (See what happens when we add non-white noise)



## 11/09/2017

Meeting with Christian. Very small difference for 1 bit case is very interesting. Investigate further by making the following adjustments for the next meeting:

* Same Seed for Noise and for CMB $\rightarrow$ saves time and makes comparison easier (also makes code tidier). $\checkmark$ Done this for CMB, as much as possible for noise
* Get code running on cloud spt (send SSH key and username to Christian) $\checkmark$ Done
* Investigate difference between powerspectra in terms of ratio $\checkmark$ For 5 observations average ratio is $\approx 97\%$

<img src = "../cosmologydigitisation/Plots/19092017/psratio.png">

* Test how difference evolves with N Observations $\checkmark$ Looks like $>20$ observations does not improve accuracy anymore

Difference in Powerspectra
<img src = "../cosmologydigitisation/Plots/19092017/psdiff.png">

Average Difference with $N_{obs}$
<img src = "../cosmologydigitisation/Plots/19092017/avgdiffnobs.png">

* Look at errorbars of powerspectrum (probably fine but still) $\checkmark$ Average error ratio is $\approx 98.7\%$ for 5 observations (digitised higher)

## 19/09/2017

Meeting with Christian. Ratio plot is most interesting. For the next meeting investigate:

* Different Digitisation Schemes (and plot as ratios) $\checkmark$ Done. Results below. Note that the equal number scheme had to be discarded, since when averaging the tod we will get zero.

<img src = "../cosmologydigitisation/Plots/24092017/schemesmaps.png">

<img src = "../cosmologydigitisation/Plots/24092017/schemesmapsdiff.png">

<img src = "../cosmologydigitisation/Plots/24092017/psdigitschemes.png">

<img src = "../cosmologydigitisation/Plots/24092017/psdigitschemesratiozoomshaded.png">

Notice how 1Bit is very close to the optimal 2Bit digitisation scheme. Notice how all schemes stay within $1\%$ up until $\approx 3\times 10^3$ and within $5\%$ up until $\approx 6 \times 10^3$. 1Bit and 2Bit opt are more well-behaved than 2Bit hm.

* Read Maris et al. 2003 "The effect of signal digitisation in CMB experiments" $\checkmark$ They focus on Satellite Transmission limitations. Key process: Quantisation, Reconstruction, Averaging (QRA). Focus seems to be on general digitisation, no outline of digitisation methods, bits etc. More likely to deal with 64/32bit rather than extreme compression. However because we need to normalise the tod we are in the regime $\sigma_T/q \approx 1$, which the paper touches upon.

* Optional: Non-white noise of form $((\frac{l_{knee}}{l})^{8/3}+1)C_{l}^{N}$ with $l_{knee} = 1000$ NOT DONE

## 25/09/2017

Meeting with Christian. Interesting how close 1Bit and 2Bit optimal schemes evolve. Let's see if the above behaviour relaxes a bit if we average over 100 CMB seeds. To do this run on cloud with multi-processors.

* Import 3G environment (batch script?)
* use mpirun
* get pythonscript that generates the cmb seeds and then calls usual script

Propose use of mpi4py, would make things a little more straightforward, already installed.

* Environment already present on cloud without any further setup
* IO should be okay. Have 4 output maps per seed, have 100 seeds $\approx 500$ maps. Each map takes about 7MB, therefore need around 3.5GB, have approx. 80GB available
* Should be able to take 12 cores for a day without stepping on someones toes

$\checkmark$ Currently running on cloud, completed 10 seeds for 12 ranks, getting a smoothed out results. running again to get even more data.

Ran over 240 seeds in total. Comparison below to a single seed to see smoothed out behaviour.

<img src = "../cosmologydigitisation/Plots/25092017/psratio1seed240seeds.png">

Now focus on the many seeds and observe how all is within $0.1\%$ until $k \approx 3-4 \times 10^3$. 1 bit and 2 bit optimal still follow each other closely, but 2 bit optimal has best performance.

<img src = "../cosmologydigitisation/Plots/25092017/manyseedsratio.png">


## 03/10/2017

Meeting with Christian. The remaining features in the smoothed out ratio are most likely still due to sample size. Ratio going off in the end probably due to the low value of the powerspectrum. For the next meeting:

* Read about TEB-IQU in mapmaking section of thesis
* Schedule availability on astro group doodle poll to have a talk in October-December time $\checkmark$ Volunteered for December 4th
* Would also like to investigate PSVL issues and non-white noise effects

Also mentioned:
* Literature Review in March time, also discuss thesis structure then
* Another talk in May-July time
* Complete Advanced Seminar before the end of next Semester

Additionally:
* Going well with research, have 3 weeks off starting after next meeting to focus on exams & assignments

## 16/11/2017

Meeting with Christian. Start Research back up. Focus will now be:

* Non-white noise
* Cluster-finder
* Polarisation

For now non-white noise. Make a PSD for noise, then into 2D Map fourier, then real, then add to TOD data. Use $(1+(k/k_{knee})^{-8/3})$ like form for a start. Place $k_{knee}$ around 100-1000 so that we get a dynamic range around it and can figure out qualitatively what happens left, around and right of it.

## 22/12/2017

Meeting with Christian. Go to AIP. Okay to take time off for nationals. Uni will pay for registration and accomodation and travel difference if reasonable.

Fix non-wn PS by maybe not cutting map into a quarter before transforming? Otherwise move to doing it as a long 1D array that goes along the scan strategy. Therefore also non-wn in time (okay for 1 detector approx.).

Show Christian presentation on Friday.

## 27/12/2017

Presentation at the group meeting. Went well. A few pointers:

* Need to get better at answering questions
* More on motivation
* Units on noise

Do a more thorough analysis on noise now. Seems rather high. Check with observation number. Check for different digitisation schemes.

## 01/12/2017

Done a more detailed review of Parseval's theorem. Tracing power through the code.

* Start out with good Powerspectrum provided by $D_l$
* Need to understand how this relates to the power in 2D array of $C_l$
* Unclear how the factor $\sqrt{\frac{df}{2}C_l}$ conserves power or restores it
* Power of cmb freq map is not the same as factor, because

$ \sum_i |f_i \times [N(0, 1) + iN(0, 1)] |^2 = \sum_i |f_i|^2 [N(0, 1)^2 + N(0, 1)^2] \neq \sum_i |f_i|^2$

* After this however more straightforward. Frequency space power $\times 1/A = $ real space power after ifft
* Cutting the map into a quarter roughly cuts the power into a quarter
* Disregarding the imaginary part of the real space map again roughly halves the power
* Zeropadding adds no power but must be careful to adjust wavevector appropriately
* Masking introduces a factor of roughly mean(mask^2) but this is nowhere near accurate
* Power preserved in PSD, need to adjust mapparameters for appropriate zeropadding
* Power preserved through radial averaging

Therefore cannot obtain correct scaling on y axis at the moment. Do get peaks at correct $l$. Relative strength of peaks also ok. Need to convene with Christian about these issues.

With regard to map size: At the moment can get down to $l = 5.3$

Noise level: Want something like $2.5\mu K^2$ across the entire survey. Hence for each observation need to have $\times \sqrt{N_{obs}}$. But surely this is across each map. how does this translate to a TOD level? Surely need to multiply again by $\times \sqrt{N_{compression}}$

Now set digitisation levels arbitrary and power normalise maps and look at the resulting powerspectra. Levels just don't seem arbitrary. Multiplying the final map with a constant factor seems challenging. Means we introduce A LOT of power. Need to look at this further.

<img src = "../cosmologydigitisation/Plots/14122017/inputps.png">

<img src = "../cosmologydigitisation/Plots/14122017/observedps.png">

<img src = "../cosmologydigitisation/Plots/14122017/cmbdigitmap.png">

<img src = "../cosmologydigitisation/Plots/14122017/cmbdigitps.png">


## 15/12/2017

Finished tracing the power. We do the following operations to the map:

* iFFT
* Cut into a quarter
* Take real part
* Mask
* Zero-pad
* FFT
* Radial Average

Which can be accounted for in power as:

* $\times N^2$
* $\times 4$
* $\times 2$
* $\times \frac{N^2}{\sum_i w_i^2}$
* $\times 1$
* $\times \frac{1}{N^2}$
* $\times \mathrm{hits}$

In summary we can correct for the power in the powerspectrum as

$$ \mathrm{PowerFrequencyMap} = 8 \times N \times N \times \mathrm{hits} \times \mathrm{Powerspectrum} $$

where $N$ is the original number of pixels, i.e. $1024$

To now compare to the factor $\sqrt{\frac{df}{2}C_l}$ we need to introduce a factor of $1/2$, since we multiply by two $N(0, 1)$ sequences to get a frequency space realisation.

This gives the correct power conversion between factor and powerspectrum as shown by the plot below. However we are still missing the input powerspectrum by a small figure. Notice that this is non-constant?! Maybe analysis w.r.t. $l$ is necessary. We think that our formulation of the factor misses some power. We exclude the idea that we are missing an additional contribution in the factor. We redo the CL2D calculation to make sure there is no error in here. $df$ seems correct.

Following plots for 500 realisations.

<img src = "../cosmologydigitisation/Plots/19122017/powfactorps.png">

<img src = "../cosmologydigitisation/Plots/19122017/psratio500runs.png">



## 20/12/2017

Meeting with Christian. Check white noise to see if I am just missing a factor or if I'm pushing power into lower modes. Also change map size and resolution to see if this induces a change to see whether I'm just missing a factor or what is going on.

Over 500 runs have found $1.192\pm0.008$ as a factor. not really sure what this matches....
Factor fixes the spectrum for now. what happens at low l? Below Nyquist? Using $l\sim \frac{180 deg}{\theta}$ and we have a map size of $1024$ arcminutes we cna get down to something like $l \sim 10$.
Get large power at low l starting at around 60 however. this is well above the Nyquist. Change the map map parameters to be sure.

<img src = "../cosmologydigitisation/Plots/22122017/psfacadj.png">

Compare Real Space Map Parameters.

1.) Fieldsize: 1024 arcminutes, resolution: 2 arcminutes

2.) Fieldsize: 2048 arcminutes, resolution: 2 arcminutes

3.) Fieldsize: 1024 arcminutes, resolution: 1 arcminute

Fixed issue with creating CL2D code taking too long, but now get nearly constant CL2D. Trying to figure out why this is. Maybe need slightly different map parameters to sample provided PS properly. Not really... Always get an exponentially rising PS, no structure...

<img src = "../cosmologydigitisation/Plots/22122017/psexpo.png">



25/01/2018

Some underlying chessboard pattern in map. Investigate creation of map further. White noise works.... No need to use fftshift when creating cl2d!!! fixed the powerspectrum!

Now get correct shapes, but 1 and 4 resolutions are squeezed/stretched by a factor of 0.5 and 2 along the horizontal respectively...

PS code was overwriting the correctly passed resolution information. changed this. PS scale looks good. Now focussing on height. res 1 is a factor of 4 too small. res of 4 is a factor of 4 too big. Found an old inflexible adjustment (1024 instead of pixelnumber) - changed this, fixed the height of the PS.

Next change the field size to see if the code still gets the correct power.

Changing the field size seems to introduce a simple factor of 2 for going to 1024 and 0.5 for going to 4096 (both from 2048). not sure where this comes from, but can easily be adjusted for. Is this okay to do? We are doing some magic number adjustment before soooo? But this is field size dependent... NOT resolution.... maybe somewhere multiply by fieldsize when it should be map pixelnumber?! No this is not being done...

30/01/2018

Produced plots for all fieldsize/resolution combinations showing the corrected powerspectrum. Need to clarify whether this adjustment is justified in meeting tomorrow and look into what next, scan strategy ok? Induced noise at correct level? Noise creation mechanism?